In [1]:
import pickle
import pandas as pd
import numpy as np

import random
from IPython.display import clear_output

In [5]:
DATA_PATH = '../../3.data/BERT/'
JOBKORRESUME_PATH = 'JobKorea_SplitedSentence.pickle'
JOBKORFINAL_PATH = 'JobKor_SplitedSentence_final.pickle'

# 옳은 문장 데이터 셋 구성

## 자기소개서 데이터 Load

In [ ]:
f = open(DATA_PATH + JOBKORRESUME_PATH, "rb")
processed_contents = pickle.load(f)
f.close()

processed_contents.head(2)

In [9]:
f = open(DATA_PATH + JOBKORFINAL_PATH, 'rb')
splitData_final = pickle.load(f)
f.close()

## Make Pair Data - 1 Label

In [13]:
correctLabel = []

for contents in splitData_final:
    for content in contents:
        for paragraph in content:
            for i, sentence in enumerate(paragraph[:-1]):
#                 try:
                nextSentence = paragraph[i+1]
                correctLabel.append((sentence, nextSentence, 1))
#                 except IndexError as e:
#                     correctLabel.append((sentence, "", 1))
                    
print(len(correctLabel))
correctLabel[:3]

162831


[('새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.',
  '조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.',
  1),
 ('조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.',
  '잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.',
  1),
 ('잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.',
  '저는 좌절보다는 노력을 택하였습니다.',
  1)]

In [4]:
## df로 할 경우
# def correctLabeling(contentsList):
#     correctDataset = []
    
#     for content in contentsList:
#         for i in range(len(content[:-1])):
#             # 문장, 다음문장, label
#             correctDataset.append((content[i], content[i+1], 1))
            
#     return correctDataset

# correctLabel = []

# for contents in processed_contents['split_content']:
#     correctLabel.extend(correctLabeling(contents))
    
# correctLabel = np.array(correctLabel)
# print(correctLabel.shape)
# correctLabel[:2]

### 데이터 저장

In [6]:
f=open("Correctdata.pickle", 'wb')
pickle.dump(correctLabel, f)
f.close()

## Make Pair Data - 0 Label 

* 옳은 문장 데이터 총 개수 : 223521개
* 틀리 문장 데이터 총 개수 : 223521개로 구성해야함 - 옳은 문장 데이터와 같이
    * 다른 자소서의 랜덤 문장과 매치 : 74507
    * 한국어 대화 문장에서 질문과 매치 : 74507
    * 한국어 대화 문장에서 답변과 매치 : 74507

### Make Type1 (자소서 문장, 다른 랜덤 자소서 문장)

In [14]:
sentenceOfDoc = []

for contents in splitData_final:
    contentTemp = []
    for content in contents:
        for paragraph in content:
            contentTemp.extend(paragraph)
        
    sentenceOfDoc.append(contentTemp)
    
len(sentenceOfDoc)

# for contents in processed_contents['split_content']:
#     temp = []
    
#     for content in contents:
#         temp.extend(content)
        
#     sentenceOfDoc.append(temp)

6760

In [15]:
print("이전: ", len(sentenceOfDoc))
flage =True
while flage:
    try:
        sentenceOfDoc.remove([])
    except ValueError:
        print('end')
        flage = False
print("이후: ", len(sentenceOfDoc))

이전:  6760
end
이후:  6712


In [16]:
inCorrectResumedata = []

while len(inCorrectResumedata) < (len(correctLabel) // 3) + 1:
    
    doc1_idx = random.randrange(len(sentenceOfDoc))
    doc2_idx = random.randrange(len(sentenceOfDoc))

    if doc1_idx != doc2_idx:
        sentencesOfDoc1 = sentenceOfDoc[doc1_idx]
        sentencesOfDoc2 = sentenceOfDoc[doc2_idx]
        sentence1_idx = random.randrange(len(sentencesOfDoc1))
        sentence2_idx = random.randrange(len(sentencesOfDoc2))
        
        # '' 아무것도 없는 문장은 넣지 않기
        if len(sentencesOfDoc1[sentence1_idx]) != 0 and len(sentencesOfDoc2[sentence2_idx]) != 0:
            inCorrectResumedata.append((sentencesOfDoc1[sentence1_idx], sentencesOfDoc2[sentence2_idx], 0))
        if len(inCorrectResumedata) % 100 == 0:
            print("현재 데이터 쌍 개수(Resume): {}".format(len(inCorrectResumedata)), end='\r', flush=True)

print("최종 데이터 쌍 개수(Resume): {}".format(len(inCorrectResumedata)))
inCorrectResumedata[:3]
# 중복 제거 해보기

최종 데이터 쌍 개수(Resume): 54278


[('철저한 기초조사와 환경 분석을 바탕으로 시각적이고 기능적인 측면에 치중하기 쉬운 디자인을 환경적. 생태적 측면에서도 종합적으로 고려해 건축물 및 여러 시설과 조화를 이루어 내는 환경을 만들겠습니다.',
  '분명 Date나 Time을 받을 때는 문제가 없었지만 datetime-local를 사용할 때에는 오류가 발생하는 것이었습니다.',
  0),
 ('공정개발 직무는 양산에 있어 최고의 수율을 내기 위해 공정개발과 공정조건의 표준화, 모니터링을 통한 품질관리를 하는 업무입니다.',
  '적극적으로 고객의 니즈를 파악하고 이를 실현하기 위해 노력하며 그 과정에서 진정으로 고객들이 가진 꿈이 무엇인지 알아내는 빙그레의 영업인이 되겠습니다.',
  0),
 ('저는 아버지 사업으로 인해 중국에서 고등학교를 1년 다니게 되면서 학년을 낮춰서 학교를 다니게 되었습니다.',
  '2학년 학생들의 성적표를 분석해야 하다 보니 양이 많았고 혼자 힘으로는 역부족이라 생각해 3명이서 분담했습니다.',
  0)]

### Make Type2, 3 (자소서 문장, 한국어 대화 중 질문 or 답변 문장)

#### 외부데이터에서 문장 가져오기

In [17]:
dialog_1 = pd.read_excel('data/Korean_dialog_data/academy(4,773).xlsx', engine='openpyxl')
dialog_2 = pd.read_excel('data/Korean_dialog_data/accommodation(7,113).xlsx', engine='openpyxl')
dialog_3 = pd.read_excel('data/Korean_dialog_data/cafe(7,859).xlsx', engine='openpyxl')
dialog_4 = pd.read_excel('data/Korean_dialog_data/closet(15,826).xlsx', engine='openpyxl')
dialog_5 = pd.read_excel('data/Korean_dialog_data/lifeService(11,087).xlsx', engine='openpyxl')
dialog_6 = pd.read_excel('data/Korean_dialog_data/realEstate(8,131).xlsx', engine='openpyxl')
dialog_7 = pd.read_excel('data/Korean_dialog_data/Restaurant(15,726).xlsx', engine='openpyxl')
dialog_8 = pd.read_excel('data/Korean_dialog_data/retailStore(14,949).xlsx', engine='openpyxl')
dialog_9 = pd.read_excel('data/Korean_dialog_data/tourLeisureEntertainment(4,949).xlsx', engine='openpyxl')

dialogs = [dialog_1, dialog_2, dialog_3, dialog_4, dialog_5, dialog_6, dialog_7, dialog_8, dialog_9]

In [18]:
dialog_9.head(2)

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,1시간에 얼마인가요?,H,관광여가오락,pc방,1,1,이용 가격 문의,NaN,Q,NaN,1시간에 얼마인가요?,NaN,NaN,NaN,1시간,NaN,1 시간/이용시간
1,점원,처음 1시간은 1000원이고 이후 1시간은 500원씩 추가됩니다,H,관광여가오락,pc방,0,2,이용 가격 문의,NaN,A,NaN,NaN,NaN,NaN,처음 1시간은 1000원이고 이후 1시간은 500원씩 추가됩니다,"처음, 1시간, 1000원, 이후, 1시간, 500원, 추가",NaN,"1시간/이용시간, 1000원/이용가격, 1시간/이용시간, 500원/이용가격"


In [19]:
Qdata = []
Adata = []

# 질문, 답변 추출
for i, dialog in enumerate(dialogs):
    for sentence in dialog[dialog.QA == 'Q']['SENTENCE']:
        Qdata.append(sentence)
        
    for sentence in dialog[dialog.QA == 'A']['SENTENCE']:
        Adata.append(sentence)
        
# 중복제거
Qdata = list(set(Qdata))
Adata = list(set(Adata))
len(Qdata), len(Adata)

(47148, 32832)

In [20]:
# 자기소개서 문장 리스트
sentencesOfResumes = []

for contents in splitData_final:
    contentTemp = []
    for content in contents:
        for paragraph in content:
            contentTemp.extend(paragraph)
        
    sentencesOfResumes.extend(contentTemp)

sentencesOfResumes = list(set(sentencesOfResumes))    
len(sentencesOfResumes)

224660

#### 자소서 문서와 한국어 대화 질문or답변 문장 pair index 데이터 구축

이전에 계속해서 랜덤 숫자를 입력하여 할 경우 중복되는 경우의 수가 자주 발생하여 **시간이 오래**걸린다

**대안**\
**조합**을 이용하여 자기소개서와 한국어 대화 질문 or 답변 문장의 모든 경우의 수를 구한뒤 원하는 개수만큼 랜덤 추출을 한다.\
실행 중간 memory부족이 발생하여 데이터 나누어서 실행

In [21]:
sentenceIdx_Q = []

# 자소서 문장 n개 랜덤 index 생성 (질문 문장이랑 Pair)
while len(sentenceIdx_Q) < (len(correctLabel) // 3) + 1:
    randIdx = random.randrange(len(sentencesOfResumes))
    if randIdx not in sentenceIdx_Q:
        sentenceIdx_Q.append(randIdx)
        if len(sentenceIdx_Q) % 100 == 0:
            print(len(sentenceIdx_Q), end='\r', flush=True)
            
print("Complet 한국어 대화 질문 랜덤용 자소서 index : {}개".format(len(sentenceIdx_Q)))
            
sentenceIdx_A = []

# 자소서 문장 n개 랜덤 index 생성 (답변 문장이랑 Pair)
while len(sentenceIdx_A) < len(correctLabel) // 3:
    randIdx = random.randrange(len(sentencesOfResumes))
    if randIdx not in sentenceIdx_A:
        sentenceIdx_A.append(randIdx)
        if len(sentenceIdx_A) % 100 == 0:
            print(len(sentenceIdx_A), end='\r', flush=True)
            
print("Complet 한국어 대화 답변 랜덤용 자소서 index : {}개".format(len(sentenceIdx_A)))

Complet 한국어 대화 질문 랜덤용 자소서 index : 54278개
Complet 한국어 대화 답변 랜덤용 자소서 index : 54277개


In [22]:
# 한국어 대화 질문 문장 n개 랜덤 index 생성
dupliQNum = (len(correctLabel) // 3) // len(Qdata)

QIdxList = list(range(len(Qdata))) * dupliQNum
QIdx = []
wantQDataNum = (len(correctLabel) // 3 + 1) - (len(Qdata) * dupliQNum)

while len(QIdx) < wantQDataNum:
    randIdx = random.randrange(len(Qdata))
    if randIdx not in QIdx:
        QIdx.append(randIdx)
        if len(QIdx) % 100 == 0:
            print(len(QIdx), end='\r', flush=True)
            
QIdxList.extend(QIdx)
print("Complet 한국어 대화 질문 랜덤 index : {}개".format(len(QIdxList)))

# 한국어 대화 답변 문장 n개 랜덤 index 생성
dupliANum = (len(correctLabel) // 3) // len(Adata)

AIdxList = list(range(len(Adata))) * dupliANum
AIdx = []
wantADataNum = (len(correctLabel) // 3) - (len(Adata) * dupliANum)

while len(AIdx) < wantADataNum:
    randIdx = random.randrange(len(Adata))
    if randIdx not in AIdx:
        AIdx.append(randIdx)
        if len(AIdx) % 100 == 0:
            print(len(AIdx), end='\r', flush=True)
            
AIdxList.extend(AIdx)
print("Complet 한국어 대화 답변 랜덤 index : {}개".format(len(AIdxList)))

Complet 한국어 대화 질문 랜덤 index : 54278개
Complet 한국어 대화 답변 랜덤 index : 54277개


In [23]:
random.shuffle(QIdxList)
random.shuffle(sentenceIdx_Q)

inCorrectQdata = []

for i, j in zip(sentenceIdx_Q, QIdxList):
    inCorrectQdata.append((sentencesOfResumes[i], Qdata[j], 0))
    
print("Complet making incorrect pair data - (자소서 문장, 한국어 대화 답변)  : {}개".format(len(inCorrectQdata)))
print(inCorrectQdata[:3])

Complet making incorrect pair data - (자소서 문장, 한국어 대화 답변)  : 54278개
[('친구들과 용돈과 추억이란 두 마리 토끼를 잡기 위해서, 신촌 즉석 사진 찍기 아르바이트를 했습니다.', '4명 김치 전골 시킬건데 할인 쿠폰 사용가능하죠?', 0), ('다양한 측면에서 사고하는 힘을 발휘해 한미약품의 미래 성장 동력을 발굴하겠습니다.', '현금영수증도 할게요', 0), ('고3 수능시험을 본 후 지원했던 학교 모두 탈락했을 때가 제가 가장 좌절했던 순간이 아닐까 생각합니다.', '토지 100평 정도 사고 싶은데 있나요?', 0)]


In [24]:
# 순서 섞어주기
random.shuffle(AIdxList)
random.shuffle(sentenceIdx_A)

inCorrectAdata = []

for i, j in zip(sentenceIdx_A, AIdxList):
    inCorrectAdata.append((sentencesOfResumes[i], Adata[j], 0))
    
print("Complet making incorrect pair data - (자소서 문장, 한국어 대화 답변)  : {}개".format(len(inCorrectAdata)))
print(inCorrectAdata[:3])

Complet making incorrect pair data - (자소서 문장, 한국어 대화 답변)  : 54277개
[('이 과정에서 시장 수요예측부터 자재 소요계획, 조달 전략까지 공부하면서 기업 내외부의 자원 흐름과 재고 감축 방안에 관해 고민할 수 있었습니다.', '앞쪽에 핸드폰 번호 찍어주시면 포인트 확인해드릴게요', 0), ('이제는 가치 창조와 미래를 선도하는 행복한 경신을 만들어 나가는 도전에 과감히 행동으로 옮기는 인재가 될 것입니다.', '지금 이게 안되요', 0), ('그렇게 적극적인 자세와 뜨거운 열정을 지닌 채 맡은 직무뿐만 아니라 다방면에 능통한 전문가로 탈바꿈하여 게임으로 하나 되는 세상을 만들어가고 싶습니다.', '생크림 케이크는 18000원이에요', 0)]


In [25]:
inCorrectData = []
inCorrectData.extend(inCorrectResumedata)
inCorrectData.extend(inCorrectQdata)
inCorrectData.extend(inCorrectAdata)
random.shuffle(inCorrectData)
print("Complet making incorrect pair data: {}개".format(len(inCorrectData)))
inCorrectData[:3]

Complet making incorrect pair data: 162833개


[('사소한 것이라 생각하셔서 혹은 문제없을 것이란 이유로 본인 편의를 봐주시길 바라시는 분이 거절당하셨을 때 , 제 평판을 폄훼하시곤 하셨습니다.',
  '처음 봤을 때 아이는 부정적 태도와 거친 성향을 보이며 대화와 학습을 거부했습니다.',
  0),
 ('어렵고 힘든 일을 피하려고 생각하기보다는 해냈을 때의 성취감을 먼저 상상하는 사람입니다.', '아니요 똑같아요', 0),
 ('또한, 사물 인터넷을 항만 및 항만 배후단지와 연계한다면 정보 처리 속도를 높여 생산성 향상에도 이바지할 수 있을 것입니다.',
  '디저트는 커피가 안 되고요 녹차랑 사이다 콜라만 나옵니다',
  0)]

### 데이터 저장

In [80]:
f = open("inCorrectdata.pickle", 'wb')
pickle.dump(inCorrectData, f)
f.close()

## 모든 데이터 병합

In [26]:
len(correctLabel), len(inCorrectData)

(162831, 162833)

In [27]:
sentencePairData = []
sentencePairData.extend(correctLabel)
sentencePairData.extend(inCorrectData)
random.shuffle(sentencePairData)
print("Complet making Pair data: {}개".format(len(sentencePairData)))
sentencePairData[:3]

Complet making Pair data: 325662개


[('둘째, ICT 산학협력 SW 교에서는 오답 노트 시스템을 개발했습니다.', '네 좋아해요', 0),
 ('금융업계에서의 올해 최고 이슈는 ‘핀테크’입니다.',
  '은행을 비롯해 모든 금융업계는 IT인프라를 늘리고, 관련 부서를 신설하는 등 핀테크에 큰 관심을 쏟고 있습니다.',
  1),
 ('이러한 열정과 도전정신은 저의 인생 모토가 되었고 어떠한 위기와 역경에도 일단 부딪혀 보며 해결책을 찾고자 하였습니다.',
  '학생회를 통해 리더십과 책임감을 발휘할 수 있었으며 혼자가 아닌 팀으로 하여 더 효율적이며 강한 팀워크를 발휘할 수 있었습니다.',
  0)]

In [ ]:
# DataFrame으로 만들기
df = pd.DataFrame(sentencePairData, columns=['first_sentence', 'second_sentence', 'label'])
df.head()

In [34]:
df[df.first_sentence.str.len() >  400]

,first_sentence,second_sentence,label
46400,"여행은 사람이 살아가는 데 있어서 출장, 신혼여행, 가족여행 등의 다양한 목적으로 ...",조경기사와 자연생태복원기사를 취득함으로써 환경 분야의 기술기초이론과 실무지식을 쌓았...,0
199092,3년간의 영업사원을 통해 거리에서 시작해 우수사원이 되기까지 많은 고객을 상대하며 ...,등이 있습니다.,0


In [35]:
df[df.first_sentence.str.len() == 1]

,first_sentence,second_sentence,label
24831,＞,"파운드리 업체의 공정 사고는 고객 신뢰를 떨어뜨릴 수 있으며, 작은 부주의가 큰 사...",0
25449,주,네 맞을 것 같아요,0
93207,원,테이크아웃 컵이 하나 더 필요한데 금액이 추가되나요?,0
133462,2,1호가 얼마에요?,0
157698,그,"또한, 활발한 소통이 이루어지는 조직일 수록 성과가 높고 갈등 해결이 원활하다는 것...",0
159645,그,1개에 1000원인데 단골이시니까 5000원에 드릴게,0
173848,유,"당시 저는 서장님의 운전병 직무를 맡아 비서실에서 근무하였고, 제 동기는 구조대에 ...",0
207370,.,휴학하기 전 마케팅 연수를 듣고 `컴퓨터공학도가 마케터처럼 아이디어가 많으면 얼마나...,0
211035,-,가게 언제 여나요?,0
271288,`,"네, 수수료 부담하셔야 합니다",0


### 데이터 저장

In [52]:
df.to_csv("sentencePairData.csv")

## 기타

### 토크나이즈 후 최대길이 확인

In [2]:
DATA_PATH = '../../3.data/BERT/'
PAIRDATA_PATH = 'sentencePairData.csv'
data_df = pd.read_csv(DATA_PATH + PAIRDATA_PATH, index_col=0)
data_df.columns = ['first_sentence', 'second_sentence', 'label']
# data_df = data_df.astype({'first_sentence':'str', 'second_sentence':'str'}) # 숫자로된 데이터를 문자열로 변경
data_df.head()

,first_sentence,second_sentence,label
0,"둘째, ICT 산학협력 SW 교에서는 오답 노트 시스템을 개발했습니다.",네 좋아해요,0
1,금융업계에서의 올해 최고 이슈는 ‘핀테크’입니다.,"은행을 비롯해 모든 금융업계는 IT인프라를 늘리고, 관련 부서를 신설하는 등 핀테크...",1
2,이러한 열정과 도전정신은 저의 인생 모토가 되었고 어떠한 위기와 역경에도 일단 부딪...,학생회를 통해 리더십과 책임감을 발휘할 수 있었으며 혼자가 아닌 팀으로 하여 더 효...,0
3,업무일지 작성으로 사내 인력과 상황에의 대처능력과 조직적응력을 빠르게 나타내겠습니다.,"선수들이 거세게 항의를 하면 판정을 번복하기도 하고, 스로인 반칙을 보고도 그냥 넘...",0
4,카드는 여행 구성에 상관없이 반드시 쓰는 혜택을 구성하고 O2O 서비스를 통해 고객...,신한 판클럽은 야 놀자 하고 제휴는 맺었지만 여행지나 렌터카 등 부가적인 서비스는 ...,1


In [6]:
for i in range(len(data_df)):
    if not isinstance(data_df.iloc[i]['first_sentence'], str):
        
        data_df.loc[i, 'first_sentence'] = str(data_df.iloc[i]['first_sentence'])
        
    if not isinstance(data_df.iloc[i]['second_sentence'], str):
        print("s",i)
        print(df.iloc[i].second_sentence)
        data_df.loc[i, 'second_sentence'] = str(data_df.iloc[i]['second_sentence'])

In [7]:
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [9]:
sentencePair = []
for i in range(len(data_df)):
    temp_df = data_df.iloc[i]
    sentencePair.append([temp_df['first_sentence'], temp_df['second_sentence']])
sentencePair[:3]

[['둘째, ICT 산학협력 SW 교에서는 오답 노트 시스템을 개발했습니다.', '네 좋아해요'],
 ['금융업계에서의 올해 최고 이슈는 ‘핀테크’입니다.',
  '은행을 비롯해 모든 금융업계는 IT인프라를 늘리고, 관련 부서를 신설하는 등 핀테크에 큰 관심을 쏟고 있습니다.'],
 ['이러한 열정과 도전정신은 저의 인생 모토가 되었고 어떠한 위기와 역경에도 일단 부딪혀 보며 해결책을 찾고자 하였습니다.',
  '학생회를 통해 리더십과 책임감을 발휘할 수 있었으며 혼자가 아닌 팀으로 하여 더 효율적이며 강한 팀워크를 발휘할 수 있었습니다.']]

In [10]:
encoded_dict = tokenizer(sentencePair,
                         padding=True,
                         truncation=True,
                         max_length=512,
                         return_tensors='tf')

In [159]:
rrr = tokenizer.decode(sss[0][-1])
rrr == '[ P A D ]'

True

In [11]:
sss = encoded_dict['input_ids']
for i, s in enumerate(sss):
    if not tokenizer.decode(s[-1]) == '[ P A D ]':
        print(i)

46400
59117
87750
155137
314652
